# 강화학습을 이용한 오델로(Othello)게임 학습

###### 2019102083 김연수<br><br>2019102123 임달홍

---

## 0. 오델로(Othello)란?

오델로(Othello)는 보드 게임의 한 종류이다. 리버시(Reversi)라고도 불립니다. <br><br>두 명이 가로 세로 8칸의 오델로 판 위에서 한쪽은 검은색, 다른 한쪽은 흰색인 돌을 번갈아 놓으며 진행됩니다.

![오델로](https://www.eothello.com/images/how_to_play_othello_0.png "othello")

### <게임 규칙>

* 처음에 판 가운데에 사각형으로 엇갈리게 배치된 돌 4개를 놓고 시작한다.<br><br>
* 돌은 반드시 상대방 돌을 양쪽에서 포위하여 뒤집을 수 있는 곳에 놓아야 한다.<br><br>
* 돌을 뒤집을 곳이 없는 경우에는 차례가 자동적으로 상대방에게 넘어가게 된다.<br><br>
* 아래와 같은 조건에 의해 양쪽 모두 더 이상 돌을 놓을 수 없게 되면 게임이 끝나게 된다.
> 64개의 돌 모두가 판에 가득 찬 경우 (가장 일반적)<br><br>
> 어느 한 쪽이 돌을 모두 뒤집은 경우<br><br>
> 한 차례에 양 쪽 모두 서로 차례를 넘겨야 하는 경우

* 게임이 끝났을 때 돌이 많이 있는 플레이어가 승자가 된다. 만일 돌의 개수가 같을 경우는 무승부가 된다.

출처 : https://ko.wikipedia.org/wiki/%EC%98%A4%EB%8D%B8%EB%A1%9C

---

## 1. 주제 선정 이유

오델로는 2인 제로섬 유한 확정 완전정보 게임으로, <br><br>서로가 최선의 수를 둘 경우 반드시 선수필승 혹은 후수필승 혹은 무승부의 결과가 나오며, 이론상 상대의 모든 수를 미리 읽을 수 있는 게임입니다.<br><br> 오델로를 제외하면 장기, 바둑, 오목 등이 이에 해당됩니다.<br><br>따라서, 이론상으로는 오델로도 필승법을 알아낼 수 있는 게임에 해당합니다.<br><br>하지만 대부분의 보드게임은 경우의 수가 매우 많고, 인간의 머리로는 모든 수를 읽는 것이 불가능하기에 <br><br>강화학습(Reinforcement Learning)을 활용해 최선의 수가 무엇인지 탐색해보고자 합니다.

---

## 2. 가설 정의

오델로 게임의 특성상, 현재 상대의 돌을 얼마나 많이 뒤집느냐(당장의 이득)는 크게 중요하지 않은 경우가 많다.

<img src="https://greenothello.com/content/library/ffo_guide/3_1.jpg" width="400"/>

그림 1의 네칸 남은 상황에서 흑은 단지 하나의 돌만을 가지고 있다. 그럼 백이 이기는 걸까?<br><br>
흑은 a1이나 h8 둘중에 한곳에 둘 수 있다.<br><br>
그리고 이 상황에서는 남은 칸을 모두 흑이 두게 되고(백은 둘 곳이 없기 때문에 흑의 매 수 다음에 패스가 난다),<br><br>
최종 스코어는 40-24로 흑의 승리가 된다.<br><br>
여기에서 게임이 거의 끝나갈 때 많은 돌을 가지고 있는 건 이기는데 큰 도움이 안된다는 것을 알 수 있다.<br><br>
이에 따라, 당장의 수가 뒤집는 돌의 개수(얻는 현재의 이득)보다는<br><br>
몇 수를 더 두었을 때의 수의 유리함(미래의 이득)을 예측할 수 있어야 한다는 이야기가 된다.

출처 : https://greenothello.com/library/ffo_guide3

따라서, 강화학습을 통해 수를 놓았을 때 미래에 얻을 수 있는 이득의 기댓값을 예측할 수 있게 된다면

> 강화학습을 통해 오델로 게임의 승률을 비약적으로 끌어올리거나 필승법을 알아낼 수 있지 않을까? 

라는 가설을 세울 수 있다.

---


## 3. 방향성

* 오델로의 게임 규칙에 맞게 게임 설계 및 제작<br><br>
* 오델로 게임을 플레이하고 학습할 Agent 구현<br><br>
* 학습 진행 및 결과 분석

---

## 4. 오델로 게임 제작

앞서 제시했던 오델로 게임의 규칙에 맞게 오델로 게임을 제작했습니다.<br><br>
단, 이후에 플레이 내용을 저장하고 학습할 수 있도록 구성해야한다는 점을 유념해야 했습니다.

In [ ]:
import sys
from collections import defaultdict

import numpy as np

from colorama import init, Fore, Back, Style
init(autoreset=True)


class Board(object):
    BLACK = 1
    WHITE = -1

    def __init__(self):
        self.board = np.zeros((8,8), int)
        self.board[3][3] = Board.BLACK
        self.board[4][4] = Board.BLACK
        self.board[4][3] = Board.WHITE
        self.board[3][4] = Board.WHITE

        self.remaining_squares = 8*8 - 4
        self.score = {Board.BLACK: 2, Board.WHITE: 2}

    def getScore(self):
        return self.score

    def getState(self):
        return self.board

    def isOnBoard(self, x, y):
        return x >= 0 and x <= 7 and y >= 0 and y <= 7

    def updateBoard(self, tile, row, col):
        result = self.isValidMove(tile, row, col)
        if result:
            self.board[row][col] = tile
            for row in result:
                self.board[row[0]][row[1]] = tile

            self.score[tile] += len(result) + 1

            self.score[(((tile+1)//2+1)%2)*2-1] -= len(result)

            self.remaining_squares -= 1

            return True

        else:
            return False

    def isValidMove(self, tile, xstart, ystart):
        if not self.isOnBoard(xstart, ystart) or self.board[xstart][ystart] != 0:
            return False
        self.board[xstart][ystart] = tile

        otherTile = tile * -1

        tiles_to_flip = []
        for xdirection, ydirection in ((0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)):
            x, y = xstart, ystart
            x += xdirection 
            y += ydirection 
            if self.isOnBoard(x, y) and self.board[x][y] == otherTile:
                x += xdirection
                y += ydirection
                if not self.isOnBoard(x, y):
                    continue
                while self.board[x][y] == otherTile:
                    x += xdirection
                    y += ydirection
                    if not self.isOnBoard(x, y):
                        break
                if not self.isOnBoard(x, y):
                    continue
                if self.board[x][y] == tile:
                    while True:
                        x -= xdirection
                        y -= ydirection
                        if x == xstart and y == ystart:
                            break
                        tiles_to_flip.append([x, y])

        self.board[xstart][ystart] = 0

        return tiles_to_flip

    def printBoard(self):

        def getItem(item):
            if item == Board.BLACK :
                return Fore.WHITE + "|" + Fore.BLACK + "O"
            elif item == Board.WHITE :
                return Fore.WHITE + "|" + Fore.WHITE + "O"
            else:
                return Fore.WHITE + "| "

        def getRow(row):
            return "".join(map(getItem,row))

        print("\t" + Back.GREEN +              "      BOARD      ")
        print("\t" + Back.GREEN + Fore.WHITE + " |0|1|2|3|4|5|6|7")
        for i in range(8):
            print("\t" + Back.GREEN + Fore.WHITE + "{}{}".format(i,
                getRow(self.board[i])))
            sys.stdout.write(Style.RESET_ALL)


게임판 위에 있는 돌들의 초기 위치를 설정한 후,<br><br> 게임판 위에 놓이게 될 돌(input)들이 유효한 수인지 검사하는 board.py를 작성합니다. <br><br> debug 과정에서 colorama 모듈을 통해 보드가 잘 만들어졌는지를 확인하는 시각화까지 진행했습니다.

In [ ]:
import numpy as np

import board


class Game:
    def __init__(self):
        self.players = []
        self.board = board.Board()

    def addPlayer(self, player, log_move_history = True):
        self.players.append((player, log_move_history))

    def getScore(self):
        return self.board.getScore()

    def run(self, show_board = False):
        n_passed = 0
        
        while n_passed < 2:
            n_passed = 0
            for i, player in enumerate(self.players):
                pid = i*2-1
                did_move = player[0].play(lambda r,c: self.board.updateBoard(pid,r,c),
                                   self.board.getState(), pid, player[1])

                if show_board:
                    self.board.printBoard()

                if not did_move:
                    n_passed += 1


플레이어를 추가하고 게임을 진행시키는 game.py를 작성합니다.

In [ ]:
import numpy as np
import board

class HumanPlayer:
    def play(self, place_func, board_state, me, _):
        try:
            pos = map(int, map(str.strip, input().split(" ")))
            place_func(*pos)
            return True
        except ValueError:
            return False

우선 게임이 규칙이 맞게 잘 실행되는지를 확인하기 위해 게임을 플레이할 수 있는 사람을 플레이어로 지정합니다.

In [ ]:
from game import Game
from players import *

h1 = HumanPlayer()
h2 = HumanPlayer()

g = Game()
g.addPlayer(h1)
g.addPlayer(h2)
g.run(True)


그리고 게임을 실행시켜봅니다.

![게임 실행 모습](./gamevisual.png)

입력한 좌표에 돌이 놓이며 게임이 잘 실행되는 것을 볼 수 있다.

---

## 5. 컴퓨터 플레이어 제작 및 학습

In [ ]:
import math
import numpy as np
import random
import pickle
from scipy.special import expit


class NN:
    def __init__(self, layer_dims, learning_rate):
        self.learning_rate = learning_rate
        self.layer_dims = layer_dims
        self.layers = []
        for i in range(len(layer_dims)-1):
            self.layers.append(np.random.normal(0, 1, size=(layer_dims[i+1], layer_dims[i]+1)))

        self.activation_func  = None
        self.dactivation_func = None

    def save(self, filename):
        with open(filename, "wb") as f:
            pickle.dump(self.layers, f)

    def load(self, filename):
        with open(filename, "rb") as f:
            self.layers = pickle.load(f)

    def mkVec(self, vector1D, add_bias = True):
        return np.reshape(vector1D, (len(vector1D), 1))

    def getOutput(self, input_vector):
        outputs = input_vector
        for i in range(len(self.layers)):
            outputs = activation(self.layers[i]@np.vstack((outputs, 1)))

        return outputs

    def backProp(self, sample, target):
        outputs = [sample]
        for i in range(len(self.layers)):
            outputs.append(activation(self.layers[i].dot(np.vstack((outputs[i], 1)))))


        layer_deltas = np.empty(len(outputs) - 1, object)

        layer_deltas[-1] = (target - outputs[-1]) * dactivation(outputs[-1])
        for i in range(len(layer_deltas) - 2, -1, -1):
            layer_derivative = dactivation(outputs[i+1])

            layer_deltas[i] = layer_derivative * (self.layers[i+1].T.dot(layer_deltas[i + 1])[:-1])

        for i in range(len(self.layers)):
            self.layers[i] += self.learning_rate * np.c_[outputs[i].T, 1] * layer_deltas[i]

        return outputs[-1]


def relu(x):
    return np.multiply(x > 0, x)

def drelu(x):
    return np.float64(x > 0)


def softmax(x):
    e = np.exp(x)
    return e/np.sum(e)


def activation(x):

    return np.tanh(x)

def dactivation(x):
    return 1 - np.tanh(x)**2

In [ ]:
class RLPlayer:
    def __init__(self, q_lr, discount_factor, net_lr = 0.01):
        self.policy_net = nn.NN([64, 128, 128, 64, 64], net_lr)

        self.epsilon = 0.6

        self.q_lr = q_lr
        self.discount_factor = discount_factor
        self.play_history = []
        self.wins = 0

    def play(self, place_func, board_state, me, log_history = True):
        input_state = np.apply_along_axis(lambda x: int((x==me and 1) or (x!=0 and -1)),
                                          1, board_state.reshape((64, 1))).reshape((64,1))
        made_move = False
        pos = None

        if np.random.random() < self.epsilon:
            positions = list(itertools.product(range(8), repeat = 2))
            random.shuffle(positions)
            while not made_move and positions:
                pos = positions.pop()
                made_move = place_func(*pos)

            if not made_move and not positions:
                return False

        else:
            out = self.policy_net.getOutput(input_state)
            positions = [(v,i) for i,v in enumerate(out)]
            positions.sort(key = lambda x: x[0], reverse = True)

            while not made_move and positions:
                scalar_play_point = positions.pop()[1]
                pos = scalar_play_point // 8, scalar_play_point % 8
                made_move = place_func(*pos)

            if not made_move and not positions:
                return False

        if log_history:
            self.play_history.append((np.copy(input_state), pos[0]*8 + pos[1]))

        return True

    def updateWeights(self, final_score):
        i = 0
        state, action = self.play_history[i]
        q = self.policy_net.getOutput(state)
        n_play_history = len(self.play_history)
        while i < n_play_history:
            i += 1
            if i == n_play_history:
                q[action] = final_score

            else:
                state_, action_ = self.play_history[i]
                q_ = self.policy_net.getOutput(state_)
                q[action] += self.discount_factor * np.max(q_)

            self.policy_net.backProp(state, self.policy_net.mkVec(q))

            if i != n_play_history:
                action, q = action_, q_


def OneHot(index, dim):
    a = np.zeros((dim,1))
    a[index] = 1
    return a

Q Learning을 기반으로 한 컴퓨터 플레이어 class를 만들고,<br><br> 컴퓨터 둘이 서로 othello 게임을 플레이하게 만든 뒤 이 결과를 저장합니다.

In [ ]:
from pprint import pprint

import numpy as np

from game import Game
from players import RLPlayer

player = RLPlayer(0.07, 0.99, 0.03)
rp = RLPlayer(0, 0)

match_size = 10
n_epochs = 2000

player_wins = []
for e in range(n_epochs):
    print("Epoch: %d"%e)

    player.wins = 0
    player.epsilon = (np.exp(-0.017*e)+0.11)/1.1
    player_gameplay_history = []

    for _ in range(match_size):
        print("Game: %d"%_)
        player.play_history = []

        g = Game()
        g.addPlayer(player)
        g.addPlayer(rp, False)
        g.run()

        final_score = list(g.getScore().items())
        final_score.sort()
        ttl = sum(map(lambda x: x[1], final_score))
        player_score =  (final_score[0][1]/ttl - 0.5)*2
        player.wins += player_score > 0
        print(player_score)
        player_gameplay_history.append((player.play_history, player_score))

    print(player.epsilon, player.wins)
    player_wins.append(player.wins)
    for game, score in player_gameplay_history:
        player.play_history = game
        player.updateWeights(score)

suffix = "linear-0.03"
player.policy_net.save("best-%s.weights"%suffix)
print(sum(player_wins))
with open("%d-%d-%s.csv"%(n_epochs, match_size, suffix), "w") as f:
    f.write("\n".join(map(str, player_wins)))

한 세트에 10판씩, 총 2000세트를 진행해서 학습시키고 이 결과를 저장합니다.<br><br>
각 세트의 승률은 "2000-10-linear-0.03.csv"로, <br><br>
학습 결과는 "best-linear-0.03.weights"로 저장됩니다.

## 6. 결론

In [ ]:
from game import Game
from players import *

h = HumanPlayer()
ai = RLPlayer(1,1,1)
ai.policy_net.load("best-linear-0.03.weights")

g = Game()
g.addPlayer(h)
g.addPlayer(ai)
g.run(True)

저장된 "best-linear-0.03.weights"대로 수를 두는 컴퓨터 플레이어와 사람이 플레이를 해 보았습니다.

![](hwins.png)

학습 과정을 기록한 csv파일에서도 알 수 있었지만, 학습을 거듭한다고 해서 먼저 두는 쪽의 승률이 비약적으로 상승하지는 않았습니다.<br><br> 위 사진은 AI가 흑돌로 선수를 두었지만 사람에게 패배한 경우였습니다.<br><br> AI의 특징은 유리한 수가 어디인지는 아는 것 같지만 결정적으로 승리를 가져갈수 있는 위치에 돌을 두는 방법을 잘 모르는 듯하다는 것이었습니다.<br><br>결론적으로 20000번정도의 게임 학습으로는 승률 100%의 모델을 만들 수 없었을 뿐만 아니라, <br><br>이와 같은 양상을 보아 학습을 더 진행하게 되었을 때 컴퓨터가 더 유리하게 게임을 이끌어나갈 수 있는 능력을 가질 수 있을지도 의문이었습니다.